In [34]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./example.pdf"
# it sucks a bit, PDFs handling is hard, e.g. it seems we lose even symbols like euro sign
# (it seems the pdf file itself sucks)
loader = PyPDFLoader(file_path )

docs = loader.load()

print(len(docs))

17


In [35]:
print(docs[15].page_content)
print()
print(docs[0].metadata)

ALL1
0524
!
!  
INFORMAZIONI SUGLI ONERI E SUI COSTI  
!
Di seguito sono riportate le commissioni e le condizioni 
economiche in vigore al  2 maggio 2024. Per successive 
modifiche si rinvia al sito www.directa.it. 
 
COMMISSIONI DI TRADING SUI DIVERSI MERCATI  
 
EXM (ex MTA), EGM, MIV, ETFplus, GEM  
Profili alternativi: 
•!Semplice: 5! per ordine eseguito 
•!Dinamica*: da 8 a 1,5!  
•!Variabile: 1,9 per mille per ordine eseguito, con un  
massimo di 18! e un minimo di 1,5! (il minimo è di 5! per il 
mercato GEM) per ordini fino a 500.000!   
 
ATFund  
•!Unico profilo disponibile : 1,9 per mille per ordine esegu ito, 
con un massimo di 200! e un minimo di 5! per ordini fino a 
500.000!   
  
SEDEX e EuroTLX certificati 
depositario Montetitoli 
Profili alternativi: 
•!Semplice: 6! per ordine eseguito 
•!Dinamica*: da 9 a 2,5!  
•!Variabile: 1,9 per mille per ordine eseguito, con 
massimo di 18! e minimo di 2! per ordini fino a 500.000!  
depositario Clearstream  
•!Unico profilo dis

So what just happened?

1. The loader reads the PDF at the specified path into memory.
1. It then extracts text data using the pypdf package.
1. Finally, it creates a LangChain Document for each page of the PDF with the page's content and some metadata about where in the document the text came from

Using a text splitter, you'll split your loaded documents into smaller documents that can more easily fit into an LLM's context window, then load them into a vector store. You can then create a retriever from the vector store for use in our RAG chain.

In [36]:
from langchain_ollama import ChatOllama

# it's dumb
# model = ChatOllama(
#     model="llama3.2",
# )

# it's slow, but smarter
# model = ChatOllama(
#   model="llama3.2-vision"
# )

model = ChatOllama(
  model="mistral-nemo"
)

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings

# slower but embeddings seem to be better
local_embeddings = OllamaEmbeddings(model="mxbai-embed-large")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=local_embeddings
)

retriever = vectorstore.as_retriever()

The final RAG chain:

In [42]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "Col profilo semplice, a quanto ammontano le commissioni di trading su EXM?"})

results

{'input': 'Col profilo semplice, a quanto ammontano le commissioni di trading su EXM?',
 'context': [Document(id='47747c83-c8d7-455e-8bd7-3ce722a25e71', metadata={'source': './example.pdf', 'page': 10}, page_content="poterne comprendere l’impatto effettivo sul rendimento \nconseguito.  \n \nCOMMISSIONI DI TRADING SUI DIVERSI MERCATI \nDirecta propone tre diversi tipi d i commissione: sempli-\nce, variabile, dinamica. La commissione semplice, pre-\nvista su tutti i mercati sui quali opera D irecta, è \nconveniente quando si immettono ordini di grande enti-\ntà, ma in n umero limitato nell'arco della giornata. La \ncommissione variabile è conveniente per gli ordini di \nmodesta entità. La commissione dinamica è convenien-\nte per ordini di discr eto ammontare e molto numerosi."),
  Document(id='a282f856-4695-4d2e-b1e3-347069af74ea', metadata={'source': './example.pdf', 'page': 15}, page_content='ALL1\n0524\n!\n!  \nINFORMAZIONI SUGLI ONERI E SUI COSTI  \n!\nDi seguito sono riportate le c

In [44]:
print(results["context"][0].metadata)

{'source': './example.pdf', 'page': 10}


In [47]:
rag_chain.invoke({"input": "Quanto costa una operazione fuori mercato?"})

{'input': 'Quanto costa una operazione fuori mercato?',
 'context': [Document(id='0c7f9227-6539-4af3-9caf-a60d3fa07717', metadata={'source': './example.pdf', 'page': 7}, page_content="tare la strategia di trading prescelta.  \n \n5) I covered warrant \nI covered warrant è un titolo che incorpora una opzione \nper acquistare o vende re un certo b ene in una data fu-\ntura predeterminata. È generalmente emesso da banche \no imprese di invest imento e quotato su mercati regola-\nmentati. Trattandosi di un'opzione, il portatore del tit olo \nha la facoltà, ma non l'obbligo, di c oncludere l'acquisto \no la vendita. In relazione alla natura del diritto si distin-\nguono i covered warrant di ti po call (diritto ad acquista-\nre) e quelli di tipo put (d iritto a vendere). Il diritto"),
  Document(id='d8f10d23-7b7f-4715-8673-16f36a5c407a', metadata={'source': './example.pdf', 'page': 11}, page_content='rimangono di esclusiva competenza del Cliente.  \nPer i det tagli sulle logiche e sui criter